In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame([
    ['Cola', 'S'],
    ['Cola', 'M'],
    ['Green', 'L'],
    ['Milk', 'M'],
    ], columns=['drink', 'size']
)

In [3]:
df

,drink,size
0,Cola,S
1,Cola,M
2,Green,L
3,Milk,M


In [4]:
size2int = {'S': 0, 'M': 1, 'L': 2}
df['size'] = df['size'].map(size2int)

In [5]:
df

,drink,size
0,Cola,0
1,Cola,1
2,Green,2
3,Milk,1


In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [7]:
encoder = LabelEncoder()

In [8]:
df['drinkLabel'] = encoder.fit_transform(df['drink'])

In [9]:
df

,drink,size,drinkLabel
0,Cola,0,0
1,Cola,1,0
2,Green,2,1
3,Milk,1,2


In [10]:
ohe = OneHotEncoder(sparse=False)
ohe_df = pd.DataFrame((ohe.fit_transform(df[['drink']])), columns=ohe.categories_[0])
ohe_df = ohe_df.add_prefix('drink_')
df = pd.concat([df, ohe_df], axis=1)

In [11]:
df

,drink,size,drinkLabel,drink_Cola,drink_Green,drink_Milk
0,Cola,0,0,1.0,0.0,0.0
1,Cola,1,0,1.0,0.0,0.0
2,Green,2,1,0.0,1.0,0.0
3,Milk,1,2,0.0,0.0,1.0


# 学習

In [108]:
! python train.py

Tokenization
Binary
0.8385
Count
0.8365
TF-IDF
0.8510
Bigram
0.8545


# one hotとgetdummies

In [12]:
# ! pip install featuretools

In [29]:
import featuretools as ft
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

data = ft.demo.load_mock_customer()
df = data['sessions']
df = df[['customer_id', 'device']].head(12)
df[7:8]['device'] = 'laptop'
df[8:9]['device'] = 'TV'

# Target Encodingの検証用にカラムを追加
np.random.seed(seed=3)
df['target'] = np.random.randint(1, 10, size=len(df))

In [30]:
df

,customer_id,device,target
0,2,desktop,9
1,5,mobile,4
2,4,mobile,9
3,1,mobile,9
4,4,mobile,1
5,1,tablet,6
6,3,tablet,4
7,4,laptop,6
8,1,TV,8
9,2,tablet,7


In [31]:
train = df[:7]
test = df[7:]

In [32]:
train

,customer_id,device,target
0,2,desktop,9
1,5,mobile,4
2,4,mobile,9
3,1,mobile,9
4,4,mobile,1
5,1,tablet,6
6,3,tablet,4


In [33]:
test = test.reset_index(drop=True)
test

,customer_id,device,target
0,4,laptop,6
1,1,TV,8
2,2,tablet,7
3,4,mobile,1
4,4,desktop,5


In [34]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe.fit(train[['device']])
ohe_df = pd.DataFrame((ohe.transform(train[['device']])), columns=ohe.categories_[0])
ohe_df = ohe_df.add_prefix('drink_')
train = pd.concat([train, ohe_df], axis=1)

In [35]:
train

,customer_id,device,target,drink_desktop,drink_mobile,drink_tablet
0,2,desktop,9,1.0,0.0,0.0
1,5,mobile,4,0.0,1.0,0.0
2,4,mobile,9,0.0,1.0,0.0
3,1,mobile,9,0.0,1.0,0.0
4,4,mobile,1,0.0,1.0,0.0
5,1,tablet,6,0.0,0.0,1.0
6,3,tablet,4,0.0,0.0,1.0


In [36]:
ohe_df = pd.DataFrame((ohe.transform(test[['device']])), columns=ohe.categories_[0])
ohe_df = ohe_df.add_prefix('drink_')
test = pd.concat([test, ohe_df], axis=1)

In [37]:
test

,customer_id,device,target,drink_desktop,drink_mobile,drink_tablet
0,4,laptop,6,0.0,0.0,0.0
1,1,TV,8,0.0,0.0,0.0
2,2,tablet,7,0.0,0.0,1.0
3,4,mobile,1,0.0,1.0,0.0
4,4,desktop,5,1.0,0.0,0.0


# ginzaで学習

In [8]:
import sys
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

from preprocessing import clean_html, tokenize
from utils import load_dataset, train_and_eval

sys.path.append('../../')
from tokenize_ja import tokenizer

In [9]:
# 外部モジュールを自動的にリロードする
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
x, y = load_dataset('data/amazon_reviews_multilingual_JP_v1_00.tsv', n=5000)

In [4]:
x[:3]

array(['現在、地球温暖化の悪影響が、ここまで顕在化しているとは<br />想像していませんでした。特に、このまま海面温度が上昇を<br />続けると、早晩、南極・北極の氷が大規模に溶けることに<br />よって相当の範囲の陸地が海に没し、十億人単位での難民が<br />発生するという事実には、本当に衝撃を受けました。<br /><br />ある人が、「人間は地球にすくう癌細胞のようである。<br />増殖・破壊を続け、最終的には自らも寄生先の死によって<br />滅ぶ運命にある」と言っていたことを思い出しました。<br /><br />一方、主人公であるゴア氏が、聴衆が数十人〜数百人程度の<br />世界中の教室・会議場をまわって危機を訴えるという、地道<br />な「草の根」の活動にも大変感銘を受けました。個人的には、<br />ゴア氏には、ブッシュ氏との大統領戦における、「賢いが<br />傲慢で冷徹」というイメージが強かったのですが、およそ<br />そんなことはない（＝きっと、ブッシュ陣営のネガティブ・<br />キャンペーンの影響を愚かにも受けていたのでしょう）、<br />信念を持ち、実行力が伴った、特筆すべき政治家であること<br />も分かりました。<br /><br />温暖化の危機、ゴア氏ほかの活動家・学者の行動に対する<br />不明を恥じると共に、自らもCO2ゼロ化に向けて早速行動<br />したいと思いました。<br /><br />なお、当作品は、語られる事象のすべてが具体的かつ<br />客観的に科学的根拠に支えられており、ドキュメンタリー<br />としても秀逸だと思います。<br /><br />是非ご覧になってみて下さい。お奨めします。',
       'このアクション映画ほど、男気を感じたものはあったのだろうか。<br />シンプル構成で時間をたっぷりと使った中身のあるアクション映画です。<br /><br />劇中、鉄斧を持った闇のサイコ集団（敵）が出てくるんですが<br />スタローンがこれでもかと、鉄拳制裁っ！するあたりはまさに圧巻の連続です。<br /><br />シンプルな映画ですが、家族で見ててハラハラドキドキします。<br /><br />また、この頃の映画って何か麻薬的な中毒性がありますよね。<br /

In [13]:
ginza_tokenizer_C = tokenizer.GinzaTokenizer(mode="C")

In [14]:
x = [' '.join(ginza_tokenizer_C.tokenize(text)) for text in x]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                        test_size=0.2,
                                                        random_state=42)

print('TF-IDF')
vectorizer = TfidfVectorizer()
train_and_eval(x_train, y_train, x_test, y_test, vectorizer)

TF-IDF
0.8090


In [22]:
print('Bigram')
vectorizer = TfidfVectorizer(ngram_range=(2, 2))
train_and_eval(x_train, y_train, x_test, y_test, vectorizer)

Bigram
0.7520


# Okapi

In [5]:
# print('Tokenization')
# x = [clean_html(text, strip=True) for text in x]
# x = [' '.join(tokenize(text)) for text in x]
# x_train, x_test, y_train, y_test = train_test_split(x, y,
#                                                     test_size=0.2,
#                                                     random_state=42)

In [6]:
# texts = pd.DataFrame(x_train)[0].apply(lambda x: x.split())

In [12]:
! python train_bm25.py

Tokenization
Okapi
0.8370
TF-IDF
0.8510
Bigram
0.8545
